In [ ]:
#| default_exp models.xlstm

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# xLSTM
The `xLSTM` was built to improve the `LSTM` architecture by introducing exponential gating and a modified memory structure. This implementation uses the `xLSTM` block stack to encode a timeseries, and subsequently an `MLP` decoder to create forecasts.

**References**<br>-[Maximilian Beck and Korbinian Pöppel and Markus Spanring and Andreas Auer and Oleksandra Prudnikova and Michael Kopp and Günter Klambauer and Johannes Brandstetter and Sepp Hochreiter. "xLSTM: Extended Long Short-Term Memory" (2024).](https://arxiv.org/abs/2405.04517})<br>

In [ ]:
#| hide
import logging
from fastcore.test import test_eq
from nbdev.showdoc import show_doc
from neuralforecast.common._model_checks import check_model

In [ ]:
#| export
from typing import Optional

import torch
import warnings
import torch.nn as nn

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_model import BaseModel
from neuralforecast.common._modules import MLP

try:
    from xlstm.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig
    from xlstm.blocks.mlstm.block import mLSTMBlockConfig
    IS_XLSTM_INSTALLED = True
except ImportError:
    IS_XLSTM_INSTALLED = False

In [ ]:
#| export
class xLSTM(BaseModel):
    """ xLSTM

    xLSTM encoder, with MLP decoder.

    **Parameters:**<br>
    `h`: int, forecast horizon.<br>
    `input_size`: int, considered autorregresive inputs (lags), y=[1,2,3,4] input_size=2 -> lags=[1,2].<br>
    `encoder_n_blocks`: int=2, number of blocks for the xLSTM.<br>
    `encoder_hidden_size`: int=128, units for the xLSTM's hidden state size.<br>
    `encoder_bias`: bool=True, whether or not to use biases within xLSTM blocks.<br>
    `encoder_dropout`: float=0., dropout regularization applied within xLSTM blocks.<br>
    `decoder_hidden_size`: int=128, size of hidden layer for the MLP decoder.<br>
    `decoder_layers`: int=2, number of layers for the MLP decoder.<br>
    `futr_exog_list`: str list, future exogenous columns.<br>
    `hist_exog_list`: str list, historic exogenous columns.<br>
    `stat_exog_list`: str list, static exogenous columns.<br>
    `exclude_insample_y`: bool=False, whether to exclude the target variable from the input.<br>
    `recurrent`: bool=False, whether to produce forecasts recursively (True) or direct (False).<br>
    `loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `max_steps`: int=1000, maximum number of training steps.<br>
    `learning_rate`: float=1e-3, Learning rate between (0, 1).<br>
    `num_lr_decays`: int=-1, Number of learning rate decays, evenly distributed across max_steps.<br>
    `early_stop_patience_steps`: int=-1, Number of validation iterations before early stopping.<br>
    `val_check_steps`: int=100, Number of training steps between every validation loss check.<br>
    `batch_size`: int=32, number of differentseries in each batch.<br>
    `valid_batch_size`: int=None, number of different series in each validation and test batch.<br>
    `windows_batch_size`: int=128, number of windows to sample in each training batch, default uses all.<br>
    `inference_windows_batch_size`: int=1024, number of windows to sample in each inference batch, -1 uses all.<br>
    `start_padding_enabled`: bool=False, if True, the model will pad the time series with zeros at the beginning, by input size.<br>
    `training_data_availability_threshold`: Union[float, List[float]]=0.0, minimum fraction of valid data points required for training windows. Single float applies to both insample and outsample; list of two floats specifies [insample_fraction, outsample_fraction]. Default 0.0 allows windows with only 1 valid data point (current behavior).<br>
    `step_size`: int=1, step size between each window of temporal data.<br>    
    `scaler_type`: str='robust', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `random_seed`: int=1, random_seed for pytorch initializer and numpy generators.<br>
    `drop_last_loader`: bool=False, if True `TimeSeriesDataLoader` drops last non-full batch.<br>
    `alias`: str, optional,  Custom name of the model.<br>
    `optimizer`: Subclass of 'torch.optim.Optimizer', optional, user specified optimizer instead of the default choice (Adam).<br>
    `optimizer_kwargs`: dict, optional, list of parameters used by the user specified `optimizer`.<br>
    `lr_scheduler`: Subclass of 'torch.optim.lr_scheduler.LRScheduler', optional, user specified lr_scheduler instead of the default choice (StepLR).<br>
    `lr_scheduler_kwargs`: dict, optional, list of parameters used by the user specified `lr_scheduler`.<br>    
    `dataloader_kwargs`: dict, optional, list of parameters passed into the PyTorch Lightning dataloader by the `TimeSeriesDataLoader`. <br>
    `**trainer_kwargs`: int,  keyword trainer arguments inherited from [PyTorch Lighning's trainer](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).<br>    
    """
    # Class attributes
    EXOGENOUS_FUTR = True
    EXOGENOUS_HIST = True
    EXOGENOUS_STAT = True
    MULTIVARIATE = False    # If the model produces multivariate forecasts (True) or univariate (False)
    RECURRENT = False        # If the model produces forecasts recursively (True) or direct (False)

    def __init__(self,
                 h: int,
                 input_size: int = -1,
                 inference_input_size: Optional[int] = None,
                 h_train: int = 1,
                 encoder_n_blocks: int = 2,
                 encoder_hidden_size: int = 128,
                 encoder_bias: bool = True,
                 encoder_dropout: float = 0.,
                 decoder_hidden_size: int = 128,
                 decoder_layers: int = 2,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 exclude_insample_y = False,
                 recurrent = False,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = -1,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size = 32,
                 valid_batch_size: Optional[int] = None,
                 windows_batch_size = 128,
                 inference_windows_batch_size = 1024,
                 start_padding_enabled = False,
                 training_data_availability_threshold = 0.0,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed = 1,
                 drop_last_loader = False,
                 alias: Optional[str] = None,
                 optimizer = None,
                 optimizer_kwargs = None,
                 lr_scheduler = None,
                 lr_scheduler_kwargs = None,
                 dataloader_kwargs = None,
                 **trainer_kwargs):
        
        if recurrent:
            warnings.warn("xLSTM does not support recurrent forecasts. Setting recurrent=False.")
            recurrent = False

        super(xLSTM, self).__init__(
            h=h,
            input_size=input_size,
            inference_input_size=inference_input_size,
            h_train=h_train,
            futr_exog_list=futr_exog_list,
            hist_exog_list=hist_exog_list,
            stat_exog_list=stat_exog_list,
            exclude_insample_y = exclude_insample_y,
            loss=loss,
            valid_loss=valid_loss,
            max_steps=max_steps,
            learning_rate=learning_rate,
            num_lr_decays=num_lr_decays,
            early_stop_patience_steps=early_stop_patience_steps,
            val_check_steps=val_check_steps,
            batch_size=batch_size,
            valid_batch_size=valid_batch_size,
            windows_batch_size=windows_batch_size,
            inference_windows_batch_size=inference_windows_batch_size,
            start_padding_enabled=start_padding_enabled,
            training_data_availability_threshold=training_data_availability_threshold,
            step_size=step_size,
            scaler_type=scaler_type,
            random_seed=random_seed,
            drop_last_loader=drop_last_loader,
            alias=alias,
            optimizer=optimizer,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler=lr_scheduler,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            dataloader_kwargs=dataloader_kwargs,
            **trainer_kwargs
        )

        if not IS_XLSTM_INSTALLED:
            raise ImportError(
                "Please install `xlstm`. You might also need to install `mlstm_kernels` for GPU support."
            )        

        # xLSTM input size (1 for target variable y)
        input_encoder = 1 + self.hist_exog_size + self.stat_exog_size + self.futr_exog_size

        # Architecture
        self.projection = nn.Linear(
            in_features=input_encoder,
            out_features=encoder_hidden_size
        )        
        block_stack_config = xLSTMBlockStackConfig(mlstm_block=mLSTMBlockConfig(), context_length=input_size, num_blocks=encoder_n_blocks, embedding_dim=encoder_hidden_size, bias=encoder_bias, dropout=encoder_dropout)
        self.hist_encoder = xLSTMBlockStack(block_stack_config)        

        # Decoder MLP
        self.mlp_decoder = MLP(in_features=encoder_hidden_size + self.futr_exog_size,
                            out_features=self.loss.outputsize_multiplier,
                            hidden_size=decoder_hidden_size,
                            num_layers=decoder_layers,
                            activation='ReLU',
                            dropout=0.0)
        self.upsample_sequence = nn.Linear(self.input_size, self.h)            

    def forward(self, windows_batch):
        
        # Parse windows_batch
        encoder_input = windows_batch['insample_y']                         # [B, seq_len, 1]
        futr_exog     = windows_batch['futr_exog']                          # [B, seq_len, F]
        hist_exog     = windows_batch['hist_exog']                          # [B, seq_len, X]
        stat_exog     = windows_batch['stat_exog']                          # [B, S]

        # Concatenate y, historic and static inputs              
        batch_size, seq_len = encoder_input.shape[:2]
        if self.hist_exog_size > 0:
            encoder_input = torch.cat((encoder_input, hist_exog), dim=2)    # [B, seq_len, 1] + [B, seq_len, X] -> [B, seq_len, 1 + X]

        if self.stat_exog_size > 0:
            # print(encoder_input.shape)
            stat_exog = stat_exog.unsqueeze(1).repeat(1, seq_len, 1)        # [B, S] -> [B, seq_len, S]
            encoder_input = torch.cat((encoder_input, stat_exog), dim=2)    # [B, seq_len, 1 + X] + [B, seq_len, S] -> [B, seq_len, 1 + X + S]

        if self.futr_exog_size > 0:
            encoder_input = torch.cat((encoder_input, 
                                       futr_exog[:, :seq_len]), dim=2)      # [B, seq_len, 1 + X + S] + [B, seq_len, F] -> [B, seq_len, 1 + X + S + F]

        encoder_input = self.projection(encoder_input)                    # [B, seq_len, 1 + X + S + F] -> [B, seq_len, rnn_hidden_state]
        hidden_state = self.hist_encoder(encoder_input)                 # [B, seq_len, rnn_hidden_state]
        hidden_state = hidden_state.permute(0, 2, 1)               # [B, seq_len, rnn_hidden_state] -> [B, rnn_hidden_state, seq_len]
        hidden_state = self.upsample_sequence(hidden_state)        # [B, rnn_hidden_state, seq_len] -> [B, rnn_hidden_state, h]
        hidden_state = hidden_state.permute(0, 2, 1)               # [B, rnn_hidden_state, h] -> [B, h, rnn_hidden_state]
        
        if self.futr_exog_size > 0:
            futr_exog_futr = futr_exog[:, -self.h:]                    # [B, h, F]
            hidden_state = torch.cat((hidden_state, 
                                        futr_exog_futr), dim=-1)         # [B, h, rnn_hidden_state] + [B, h, F] -> [B, h, rnn_hidden_state + F]

        output = self.mlp_decoder(hidden_state)                        # [B, h, rnn_hidden_state + F] -> [B, seq_len, n_output]

        return output

In [ ]:
show_doc(xLSTM)

In [ ]:
show_doc(xLSTM.fit, name='xLSTM.fit')

In [ ]:
show_doc(xLSTM.predict, name='xLSTM.predict')

In [ ]:
#| hide
# Unit tests for models
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("lightning_fabric").setLevel(logging.ERROR)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    check_model(xLSTM, ["airpassengers"])

In [ ]:
#| hide
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.utils import AirPassengersDF as Y_df
from neuralforecast.tsdataset import TimeSeriesDataset

## Usage Example

In [ ]:
#| eval: false
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import xLSTM
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]] # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

nf = NeuralForecast(
    models=[xLSTM(h=12, 
                 input_size=8,
                 loss=DistributionLoss(distribution="Normal", level=[80, 90]),
                 scaler_type='robust',
                 encoder_n_blocks=1,
                 encoder_hidden_size=64,
                 decoder_hidden_size=64,
                 decoder_layers=2,
                 max_steps=300,
                 futr_exog_list=['y_[lag12]'],
                 stat_exog_list=['airline1'],
                 )
    ],
    freq='ME'
)
nf.fit(df=Y_train_df, static_df=AirPassengersStatic)
Y_hat_df = nf.predict(futr_df=Y_test_df)

# Plots
Y_hat_df = Y_hat_df.reset_index(drop=False).drop(columns=['unique_id','ds'])
plot_df = pd.concat([Y_test_df, Y_hat_df], axis=1)
plot_df = pd.concat([Y_train_df, plot_df])

plot_df = plot_df[plot_df.unique_id=='Airline1'].drop('unique_id', axis=1)
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['xLSTM-median'], c='blue', label='median')
plt.fill_between(x=plot_df['ds'][-12:], 
                 y1=plot_df['xLSTM-lo-90'][-12:].values,
                 y2=plot_df['xLSTM-hi-90'][-12:].values,
                 alpha=0.4, label='level 90')
plt.grid()
plt.plot()